In [13]:
#import necessary libraries for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#for auth
import os
from dotenv import load_dotenv

#for spotify api
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [17]:
#get credentials from .env
load_dotenv()

#set up spotipy
client_id=os.getenv('SPOTIPY_CLIENT_ID')
client_secret=os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri=os.getenv('SPOTIPY_REDIRECT_URI')

scope = 'user-library-read'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                                scope=scope))

289d15da450c423aadf2ecdfdf7d2688
